In [1]:
from outils import *
from hog import hog
import numpy as np

In [2]:
D = 10
N = 100
h = hog(D,N)

In [3]:
h.jouer(h.strategie_aveugle(),h.strategie_aveugle())

(103, 80)

In [4]:
def moyenne(strategie1,strategie2,name1,name2,steps=1000):
    def f1():
        score1,score2 = h.jouer(strategie1,strategie2)
        return 0 if score1 > score2 else 1
    
    def f2():
        score1,score2 = h.jouer(strategie1,strategie2)
        return 0 if score1 > score2 else 1
    
    a = np.zeros((steps))-1
    vfunc = np.vectorize(lambda x: f1())
    s1 = 1 - vfunc(a).sum()/steps
    
    r = dict()
    t = dict()
    t[name2] = s1 
    r[name1] = t
    
    return r

def function(stratiges,names):
    from itertools import  combinations_with_replacement
    l = list(combinations_with_replacement(stratiges,2))
    names = list(combinations_with_replacement(names,2))
    
    r=list()
    for i in range(len(l)):
        name1,name2 = names[i]
        s1,s2       = l[i]
        r.append(moyenne(s1,s2,name1,name2))
        if(name1 != name2):
            r.append(moyenne(s2,s1,name2,name1))
    return r 

def get_pandas_data():
    a = [h.strategie_aveugle(),h.strategie_optimale(),h.strategie_aleatoire()]
    names=["aveugle","optimale","aleatoire"]
    a = function(a,names)
    d=dict({ n : dict() for n in names })
    for k in a:
        _k = list(k.keys())[0]
        _v = list(k.values())[0]
        default_data = dict()
        for x in k.values():
            d[_k].update(x)
    import pandas as pd
    df = pd.DataFrame(d).T
    df.to_csv("./data.csv")
    return  df

In [ ]:
import pandas as pd
p = get_pandas_data()

In [6]:
import pandas as pd

In [38]:
D=5
m = eg_simul(h.probabiltes,D)
index = [ "d="+str(i) for i in range(1,D+1)]
df = pd.DataFrame(m,index=index, columns=index)
df.to_csv("./matrice_gain.csv")

In [77]:
def result(r,joueur=1):
    v = np.random.random()
    r = np.array([np.round(v.value(),10) for v in r]).cumsum()
    r = np.where(r>v)[0][0]
    return int(r)

def roulez_les_des(d):
    from dice import dice
    assert type(d) == int, 'nombre de dés doit étre un entier.'
    assert d > 0, 'il faut faire au moin un lacer.'

    vfunc = np.vectorize(lambda x: dice())
    a = np.zeros(d) - 1
    a = vfunc(a)
    if(1 in a):
        return 1
    else:
        return a.sum()
    
def moy_simule(n=100):
    def f():
        s1 = roulez_les_des(result(r,1))
        s2 = roulez_les_des(h.strategie_aveugle()(0,0))
        if(s1>s2):
            return 1
        if(s1<s2):
            return -1
        return 0
    a = np.array(list(map(lambda x : f(),np.arange(1,n))))
    per_1 = (a==1).sum()/n
    per_2 = (a==-1).sum()/n
    per_3 = (a==0).sum()/n
    
    return np.array( [per_1,per_2,per_3])

In [90]:
r = lp_resolution(h.probabiltes,D,2)
a = moy_simule(10000)

C:\Users\yacin\Anaconda3\lib\site-packages\pulp\pulp.py:1137: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Status: Optimal
_pa = 0.0
_pb = 0.0
_pc = 0.0
_pd = 0.0
_pe = 0.0
_pf = 0.0
_pg = 0.0
_ph = 0.0
_pi = 0.0
_pj = 1.0
_z = 0.0
Total Cost of Ingredients per can =  0.0


In [89]:
pd.DataFrame(a,index=["LP_Simultané","Aveugle","Parties Nulle"],columns=["Pourcentage"]).to_csv("./resultat_Joueur1.csv")

In [92]:
pd.DataFrame(a,index=["LP_Simultané","Aveugle","Parties Nulle"],columns=["Pourcentage"]).to_csv("./resultat_Joueur2.csv")